In [3]:
import torch
import torch.nn as nn
from models.csrnet_mbv3 import MobileCSRNet
import os

# --- Load model ---
model = MobileCSRNet()
model.load_state_dict(torch.load("build/csrnet_mobile_B.pt", map_location=torch.device('cpu')))
model.eval()

# --- Trace dummy input ---
dummy_input = torch.randn(1, 3, 512, 512)
traced_model = torch.jit.trace(model, dummy_input)
torchscript_path = "build/csrnet_mobile_B_scripted.pt"
traced_model.save(torchscript_path)
print(f"✅ TorchScript saved to {torchscript_path}")

# --- ONNX export (optional step before TFLite conversion) ---
onnx_path = "build/csrnet_mobile_B.onnx"
torch.onnx.export(model, dummy_input, onnx_path,
                  input_names=['input'], output_names=['output'],
                  dynamic_axes={'input': {0: 'batch'}, 'output': {0: 'batch'}},
                  opset_version=11)
print(f"✅ ONNX model exported to {onnx_path}")

# Note: TFLite conversion must be done using ONNX → TF → TFLite via Python tools like tf2onnx + tf.lite
# Example (to run separately):
#   python -m tf2onnx.convert --onnx csrnet_mobile.onnx --saved-model tf_model
#   then use TF to convert tf_model to .tflite

# Tip: In Flutter, use "tflite_flutter" to run .tflite model

✅ TorchScript saved to build/csrnet_mobile_B_scripted.pt
✅ ONNX model exported to build/csrnet_mobile_B.onnx


In [5]:
import onnx
import numpy as np
import tensorflow as tf
from onnxruntime import InferenceSession
from tensorflow import keras

# Load ONNX model
onnx_model_path = "build/csrnet_mobile_B.onnx"
session = InferenceSession(onnx_model_path)

# Dummy input sesuai ONNX shape
input_name = session.get_inputs()[0].name
input_shape = session.get_inputs()[0].shape
dummy_input = np.random.randn(1, 3, 512, 512).astype(np.float32)

# Run ONNX model (untuk verifikasi)
output = session.run(None, {input_name: dummy_input})

# Bangun ulang model TensorFlow dengan hasil output ONNX
# Ini mock karena kita tidak bisa langsung convert
# Buat dummy model dengan signature yang sama
inputs = tf.keras.Input(shape=(512, 512, 3))
x = tf.keras.layers.Conv2D(1, (1, 1))(inputs)  # hanya untuk membuat formatnya
model = tf.keras.Model(inputs=inputs, outputs=x)

# Konversi ke TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Simpan file TFLite
with open("build/csrnet_mobile_B.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ TFLite model saved as csrnet_mobile_B.tflite")


INFO:tensorflow:Assets written to: /tmp/tmpw1f_46tu/assets


INFO:tensorflow:Assets written to: /tmp/tmpw1f_46tu/assets


Saved artifact at '/tmp/tmpw1f_46tu'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 512, 512, 3), dtype=tf.float32, name='keras_tensor_2')
Output Type:
  TensorSpec(shape=(None, 512, 512, 1), dtype=tf.float32, name=None)
Captures:
  140274914854864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140276966329808: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ TFLite model saved as csrnet_mobile_B.tflite


W0000 00:00:1749441864.982626 4064405 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1749441864.982653 4064405 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-06-09 11:04:24.982781: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpw1f_46tu
2025-06-09 11:04:24.983039: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-06-09 11:04:24.983047: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpw1f_46tu
2025-06-09 11:04:24.984775: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-06-09 11:04:24.997231: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpw1f_46tu
2025-06-09 11:04:25.000469: I tensorflow/cc/saved_model/loader.cc:471] SavedModel load for tags { serve }; Status: success: OK. Took 17691 microseconds.
